In [1]:
from scipy.io import mmread
import numpy as np
from os import listdir
from os.path import isfile, join
import pickle
import tensorflow as tf
import config
conf, _ = config.get_config()

c:\users\mmnet\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Namespace(MAX_ITERATION=10000.0, batch_size=128, debug=False, gpu=0, lr=0.0001, tar_model='model_1', training=True)


In [2]:
# put yor dataset in the dataset folder
data_dir = './dataset/'

# setting a folder for certain model
model_dir = './'+conf.tar_model+'/'

# creating the logs folder in the respective model folder
logs_dir = model_dir + 'logs/'

In [3]:
# loading the target model as model
if conf.tar_model == '[filename of your model]':
    import '''filename of your model''' as model
    print('Load', conf.tar_model)
else:
    sys.exit("Sorry, Wrong Model!")

Load model_1


# Read dataset

In [4]:
with open(data_dir+'/[filename of your dataset].pickle', 'rb') as f:
    raw_data = pickle.load(f)

In [5]:
# Have a glance at raw_data
print(type(raw_data))
print(raw_data.shape)

<class 'numpy.ndarray'>
(195000, 120, 450)


In [6]:
# You can add a preprocessing for the data
traing_data = raw_data

In [7]:
# Have a glance at traing_data
print(type(traing_data))
print(traing_data.shape)

<class 'numpy.ndarray'>
(780000, 13500)


In [8]:
# setting a batch generator for getting a batch of dataset
def batch_generator(traing_data = traing_data, batch_size = 128):
    print('Size of all training data', traing_data.shape)
    # while (1):
    idx = np.arange(0, traing_data.shape[0], 1)
    np.random.shuffle(idx)
    while(True):
        for i in range(int(len(idx)/batch_size)):
            yield traing_data[idx[i*batch_size:((i+1)*batch_size)]]

In [10]:
# defining the batch_generator
get_batch = batch_generator(traing_data, conf.batch_size) # (B, Shape of your dataset)

# Training

In [ ]:
# clear tensorflow graph
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default() as g:
    
    ###############
    # Model Setup #
    ###############
    
    # model input
    model_input = tf.placeholder(tf.float32, shape=[conf.batch_size, '''Shape of the feature for trainging'''], name="model_input")
    model_GT = tf.placeholder(tf.float32, shape=[conf.batch_size, '''Shape of the feature for trainging'''], name='model_GT')
    keep_probability = tf.placeholder(tf.float32, name="keep_probabilty")
    train_phase = tf.placeholder(tf.bool, name='phase_train')

    # build model
    code, model_out = model.inference(model_input, keep_probability, train_phase)

    # check # parameter of the model
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        #   print(shape)
        #   print(len(shape))
        variable_parameters = 1
        for dim in shape:
        #   print(dim)
            variable_parameters *= dim.value
        #   print(variable_parameters)
        total_parameters += variable_parameters
    print('Total parameters', total_parameters)
    
    # model loss
    loss = tf.reduce_mean(tf.losses.mean_squared_error(predictions=model_out, labels=model_GT))
    loss_summary = tf.summary.scalar("Tot_loss", loss)

    def train(loss_val, var_list):
        optimizer = tf.train.AdamOptimizer(conf.lr)
        grads = optimizer.compute_gradients(loss_val, var_list=var_list)
        return optimizer.apply_gradients(grads)

    trainable_var = tf.trainable_variables()
    
    train_op = train(loss, trainable_var)
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    # setup
    print("Setting up summary op...")
    summary_op = tf.summary.merge_all()

    print("Setting up Saver...")
    saver = tf.train.Saver()

    sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False), graph=g)
    
    if (conf.training == False):
        ckpt = tf.train.get_checkpoint_state(logs_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            print('Loading sucessfully')
        else:
            print('No checkpoint file found')
            raise
    else:
        init = tf.global_variables_initializer()
        sess.run(init)

    # create two summary writers to show training loss and validation loss in the same graph
    # need to create two folders 'train' and 'validation' inside FLAGS.logs_dir
    train_writer = tf.summary.FileWriter(logs_dir+'/train', sess.graph)
    validation_writer = tf.summary.FileWriter(logs_dir+'/validation', sess.graph)
    
    #########
    # Start #
    #########
    if conf.training == True:
        print("-------- Start Training --------")
        max_validloss = 99999
        for itr in range(int(conf.MAX_ITERATION)):
            
            # get training input from the generator
            batch_xs = next(get_batch)
            
            # training
            sess.run([train_op,extra_update_ops], feed_dict={model_input: batch_xs,
                                                             model_GT: batch_xs,
                                                             keep_probability: 0.85,
                                                             train_phase:conf.training})
            # print training loss per 500 updates
            if itr % 500 == 0:
                train_loss, summary_str = sess.run([loss, loss_summary], feed_dict={model_input: batch_xs,
                                                                                     model_GT: batch_xs,
                                                                                     keep_probability: 1,
                                                                                     train_phase:conf.training})
                print("[T] Step: %d, loss:%g" % (itr, np.mean(train_loss)))
                train_writer.add_summary(summary_str, itr)
            
            # if you have the validation dataset you can
            # validate the trained model per 1000 updates
#             if itr % 1000 == 0:
#                 # prepare inputs
#                 valid_losses = []
                
#                 # scan all the validation dataset
#                 for i in tqdm.trange(int('''size of your validation dataset'''/conf.batch_size)):
#                     batch_xs_valid, batch_ys_valid = next(valid_batches)
                    
#                     # getting a validation batch
#                     batch_ymap_valid = pts2map(batch_ys_valid);# print(batch_ymap_valid.shape)

#                     feed_dict = {image: batch_xs_valid,
#                                  annotation: batch_ymap_valid,
#                                  keep_probability: 1.0,
#                                  train_phase:conf.training}
#                     # feeding the validation model to the trained model
#                     valid_loss, pts_maps, summary_sva=sess.run([loss, heat_map, loss_summary], feed_dict=feed_dict)
#                     # save all the losses of validation batches
#                     valid_losses.append(valid_loss)
                    
#                 # save validation log
#                 validation_writer.add_summary(summary_sva, itr)
                
#                 # save the ckpt to logs folder if reachings better average validation loss
#                 calc_v_loss = np.mean(valid_losses)
#                 if calc_v_loss < max_validloss:
#                     saver.save(sess, logs_dir + "model.ckpt", itr)
#                     print("[V*] Step: %d, loss:%g" % (itr, calc_v_loss))
#                     max_validloss = calc_v_loss
#                 else:
#                     print("[V] Step: %d, loss:%g" % (itr, calc_v_loss))
                    
    # testing model
    else:
        print("Testing")
#         testing_batch = 2
#         inferred_map = []
#         for t in tqdm.trange(int(db_helen['img']['testset'].shape[0]/testing_batch)):
#             t_batch_x = db_helen['img']['testset'][(t*testing_batch):((t+1)*testing_batch)]
#             t_batch_y = db_helen['pts']['testset'][(t*testing_batch):((t+1)*testing_batch)]
#             feed_dict = {image: t_batch_x, keep_probability: 1.0,train_phase:conf.training}

#             batch_map=sess.run(heat_map, feed_dict=feed_dict)
#             inferred_map.append(batch_map)

#         inferred_map = np.asarray(inferred_map)
#         pts_maps = np.reshape(inferred_map, newshape=(-1,inferred_map.shape[2],inferred_map.shape[3],inferred_map.shape[4]))
#         infered_pts = map2pts(pts_maps)

#         for idx, content in enumerate(zip(db_helen['img']['testset'],infered_pts)):
#             img = content[0].copy()
#             img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#             for kp_idx, keypoint in enumerate(content[1]):
#                 cv2.circle(img,(keypoint[0],keypoint[1]), 2, (0,255,0), -1)
#                 cv2.putText(img, str(kp_idx), (keypoint[0],keypoint[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.3,(255,255,255),1,cv2.LINE_AA)

#             cv2.imwrite(pred_dir + str(idx)+ '.png', img) 

#         norm_error_image = eval_norm_error_image(infered_pts, db_helen['pts']['testset'])
#         pandas.DataFrame({'loss':norm_error_image}).to_csv(model_dir + 'norm_error_image_' + str(conf.testing)+ '.csv')
    
    # finish training
    sess.close()


total_parameters 120339388
Setting up summary op...
Setting up Saver...
Start Testing
Size of all training data (780000, 13500)
[T] Step: 0, loss:0.154082
[T] Step: 500, loss:0.00308652
[T] Step: 1000, loss:0.00382808
[T] Step: 1500, loss:0.00503836
